## 套件

In [ ]:
import pandas as pd

### 讀取 價格、財報、法人、資券資料

In [ ]:
price_data = pd.read_csv('../data/price.txt',  encoding='cp950')
ifrs_data = pd.read_csv('../data/ifrs.txt',  encoding='cp950')
institution_data = pd.read_csv('../data/institution.txt',  encoding='cp950')
margin_data = pd.read_csv('../data/margin.txt',  encoding='cp950')

# 欄位改名

In [ ]:
from utils import rename_column_maplist

In [ ]:
rename_column_maplist

In [ ]:
price_data.rename(columns=rename_column_maplist, inplace=True)
ifrs_data.rename(columns=rename_column_maplist, inplace=True)
institution_data.rename(columns=rename_column_maplist, inplace=True)
margin_data.rename(columns=rename_column_maplist, inplace=True)

# 資料前處理

# 欄位的加減乘除

In [ ]:
# 成交金額 = 收盤價 * 成交量

price_data['close'] * price_data['volume']

In [ ]:
price_data.info()

In [ ]:
# 股價淨值比
# 怎麼怪怪的

price_data['PB_TSE'] * 2

# 還有什麼欄位也有問題？

In [ ]:
price_data[['name', 'code', 'new_industry', 'industry']].values

In [ ]:
ifrs_data.head()

1. name, code, new_industry, industry 有空白
2. 有些 code 是數字
3. 部分公司的 PB_TSE, YOY, TAR 是字串
4. 營收資料是月資料

# 利用 apply 欄位資料逐步處理

In [ ]:
def testApply(value):
    print('value => ', value)
    return value

In [ ]:
price_data['name'].apply(testApply)

### 處理空白

In [ ]:
def cleanStringSpace(value):
    return str(value).strip()

In [ ]:
# 代號
price_data['code'] = price_data['code'].apply(cleanStringSpace)
institution_data['code'] = institution_data['code'].apply(cleanStringSpace)
margin_data['code'] = margin_data['code'].apply(cleanStringSpace)
ifrs_data['code'] = ifrs_data['code'].apply(cleanStringSpace)

# 名字
price_data['name'] = price_data['name'].apply(cleanStringSpace)
institution_data['name'] = institution_data['name'].apply(cleanStringSpace)
margin_data['name'] = margin_data['name'].apply(cleanStringSpace)
ifrs_data['name'] = ifrs_data['name'].apply(cleanStringSpace)

# 產業
price_data['new_industry'] = price_data['new_industry'].apply(cleanStringSpace)
price_data['industry'] = price_data['industry'].apply(cleanStringSpace)

# 處理 字串 -> 數值

In [ ]:
def StringToFloat(value):
    return float(value)

In [ ]:
# 財報
price_data['PB_TSE'] = price_data['PB_TSE'].apply(StringToFloat)
ifrs_data['YOY'] = ifrs_data['YOY'].apply(StringToFloat)
ifrs_data['TAR'] = ifrs_data['TAR'].apply(StringToFloat)

In [ ]:
price_data.head()

In [ ]:
ifrs_data.head()

#  合計週轉率 與 資券比

In [ ]:
institution_data.head()

In [ ]:
institution_data['intitution_turnover_rate'] = institution_data[['foreign_turnover_rate', 'trust_turnover_rate', 'broker_turnover_rate']].sum(1)
institution_data

# 資券比

In [ ]:
margin_data.head()

In [ ]:
def shortMarginConvert(value):
    if value == 0:
        return 0
    else:
        return 1/(value)*10000

    
margin_data['margin_short_ratio'] = margin_data['short_margin_ratio'].apply(shortMarginConvert)
margin_data

# 如何將『高頻資料』與『低頻資料』作合併

1. 配合低頻資料產生可以對應的時間欄位

In [ ]:
def dateToDateYearMonth(date_value):
    return int(str(date_value)[:6])

In [ ]:
dateToDateYearMonth(20170304)

In [ ]:
price_data['date_year_month'] = price_data['date'].apply(dateToDateYearMonth)

In [ ]:
price_data

2. 根據對應時間來合併，並保留全部資料

In [ ]:
student_data = pd.DataFrame([
    {'student_name': "John Cena", 'student_no': 1},
    {'student_name': "Mary", 'student_no': 2},
    {'student_name': "Bob", 'student_no': 3},
    {'student_name': "Patrick", 'student_no': 4},
    {'student_name': "Hank", 'student_no': 5}]
)

student_course_data = pd.DataFrame([
    {'course_name': "微積分", 'student_no': 2},
    {'course_name': "微積分", 'student_no': 5},
    {'course_name': "時間序列", 'student_no': 3},
    {'course_name': "時間序列", 'student_no': 4},
    {'course_name': "時間序列", 'student_no': 5},
    {'course_name': "程式設計", 'student_no': 6},
])

#### 合併的4方法

1. left

In [ ]:
student_data.merge(student_course_data, on='student_no', how='left')

2. right

In [ ]:
student_data.merge(student_course_data, on='student_no', how='right')

3. inner

In [ ]:
student_data.merge(student_course_data, on='student_no', how='inner')

4. outer

In [ ]:
student_data.merge(student_course_data, on='student_no', how='outer')

### price data merge ifrs data

In [ ]:
merge_data = price_data.merge(ifrs_data, 
                              on=['code', 'name', 'date_year_month'], 
                              how='outer')

In [ ]:
merge_data

In [ ]:
## 再根據時間排序一下

merge_data = merge_data.sort_values(by=['date_year_month', 'date'])
merge_data

3. 遺漏值補前一期

In [ ]:
merge_data = merge_data.ffill()
merge_data

# 把其他資料整合進來

In [ ]:
stock_data =  merge_data.merge(
    institution_data, on=['code', 'name', 'date'] ## 法人週轉率
).merge(
    margin_data, on=['code', 'name', 'date']      ## 資券比
)

In [ ]:
stock_data.columns